In [1]:
!nvidia-smi
!nvcc --version

Mon Dec 29 14:41:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%%writefile 4main.cu
// Подключаем основной заголовок CUDA Runtime
#include <cuda_runtime.h>

// Подключаем расширения для запуска CUDA-ядер
#include <device_launch_parameters.h>

// Подключаем стандартный алгоритм (stable_sort)
#include <algorithm>

// Подключаем библиотеку для измерения времени на CPU
#include <chrono>

// Подключаем ввод-вывод
#include <iostream>

// Подключаем генератор случайных чисел
#include <random>

// Подключаем контейнер vector
#include <vector>

// Используем стандартное пространство имён
using namespace std;

// Макрос для проверки ошибок CUDA
#define CUDA_CHECK(call) do {                                      \
    cudaError_t err = (call);                                      /* Выполняем CUDA-вызов */ \
    if (err != cudaSuccess) {                                      /* Если произошла ошибка */ \
        cerr << "CUDA error: "                                     /* Печатаем сообщение */ \
             << cudaGetErrorString(err)                            /* Текст ошибки CUDA */ \
             << " at " << __FILE__                                 /* Имя файла */ \
             << ":" << __LINE__ << "\n";                           /* Номер строки */ \
        exit(1);                                                   /* Аварийно завершаем программу */ \
    }                                                              \
} while(0)

// CUDA-ядро: сортировка чанков (bitonic sort)
// Каждый блок сортирует один подмассив
template<int BLOCK_SIZE>
__global__ void bitonic_sort_chunks(int* d, int n) {

    // Разделяемая память блока для чанка
    __shared__ int s[BLOCK_SIZE];

    // Индекс потока внутри блока
    int tid = threadIdx.x;

    // Начальный индекс чанка
    int blockStart = blockIdx.x * BLOCK_SIZE;

    // Глобальный индекс элемента
    int idx = blockStart + tid;

    // Если индекс в пределах массива — берём элемент,
    // иначе подставляем INT_MAX (для корректной сортировки)
    int v = (idx < n) ? d[idx] : INT_MAX;

    // Кладём значение в shared memory
    s[tid] = v;

    // Синхронизация потоков блока
    __syncthreads();

    // Классическая bitonic-сортировка
    for (int k = 2; k <= BLOCK_SIZE; k <<= 1) {
        for (int j = k >> 1; j > 0; j >>= 1) {

            // Вычисляем индекс для сравнения
            int ixj = tid ^ j;

            // Работаем только с одной парой
            if (ixj > tid) {

                // Определяем направление сортировки
                bool ascending = ((tid & k) == 0);

                // Берём значения
                int a = s[tid];
                int b = s[ixj];

                // Сортировка по возрастанию
                if (ascending) {
                    if (a > b) {
                        s[tid] = b;
                        s[ixj] = a;
                    }
                }
                // Сортировка по убыванию
                else {
                    if (a < b) {
                        s[tid] = b;
                        s[ixj] = a;
                    }
                }
            }

            // Синхронизация после шага
            __syncthreads();
        }
    }

    // Записываем отсортированные данные обратно в глобальную память
    if (idx < n) d[idx] = s[tid];
}

// CUDA-ядро: параллельное слияние двух отсортированных подмассивов
__global__ void merge_pass(const int* in, int* out, int n, int run) {

    // Номер пары подмассивов
    int pairId = blockIdx.x;

    // Начало пары
    int start = pairId * (2 * run);

    // Граница между двумя массивами
    int mid = min(start + run, n);

    // Конец пары
    int end = min(start + 2 * run, n);

    // Общая длина сливаемого диапазона
    int total = end - start;

    // Каждый поток пишет несколько элементов
    for (int t = threadIdx.x; t < total; t += blockDim.x) {

        // Длины левой и правой частей
        int leftLen = mid - start;
        int rightLen = end - mid;

        // Диапазон бинарного поиска
        int lo = max(0, t - rightLen);
        int hi = min(t, leftLen);

        // Binary search (merge path)
        while (lo < hi) {
            int i = (lo + hi) >> 1;
            int j = t - i;

            int left_i = (i < leftLen) ? in[start + i] : INT_MAX;
            int right_jm = (j > 0) ? in[mid + j - 1] : INT_MIN;

            if (right_jm > left_i)
                lo = i + 1;
            else
                hi = i;
        }

        // Финальные индексы
        int i = lo;
        int j = t - i;

        // Значения для сравнения
        int left_i = (i < leftLen) ? in[start + i] : INT_MAX;
        int right_j = (j < rightLen) ? in[mid + j] : INT_MAX;

        // Записываем минимум
        out[start + t] = (left_i <= right_j) ? left_i : right_j;
    }
}

// GPU Merge Sort (host-функция)
template<int BLOCK_SIZE>
double gpu_merge_sort(vector<int>& a) {

    // Размер массива
    int n = a.size();

    // Размер памяти в байтах
    size_t bytes = n * sizeof(int);

    // Указатели на GPU-массивы
    int* d0 = nullptr;
    int* d1 = nullptr;

    // Выделяем память на GPU
    CUDA_CHECK(cudaMalloc(&d0, bytes));
    CUDA_CHECK(cudaMalloc(&d1, bytes));

    // Копируем данные на GPU
    CUDA_CHECK(cudaMemcpy(d0, a.data(), bytes, cudaMemcpyHostToDevice));

    // CUDA-таймеры
    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));

    // Запуск таймера
    CUDA_CHECK(cudaEventRecord(start));

    // Количество блоков
    int blocks = (n + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // Сортировка чанков
    bitonic_sort_chunks<BLOCK_SIZE><<<blocks, BLOCK_SIZE>>>(d0, n);

    // Проверка ошибок
    CUDA_CHECK(cudaGetLastError());

    // Начальный размер отсортированного блока
    int run = BLOCK_SIZE;

    // Количество потоков на блок
    const int THREADS = 256;

    // Переключатель буферов
    bool ping = true;

    // Итеративное слияние
    while (run < n) {

        // Количество пар
        int pairs = (n + 2 * run - 1) / (2 * run);

        // Выбираем вход и выход
        const int* in = ping ? d0 : d1;
        int* out = ping ? d1 : d0;

        // Запуск ядра слияния
        merge_pass<<<pairs, THREADS>>>(in, out, n, run);

        // Проверка ошибок
        CUDA_CHECK(cudaGetLastError());

        // Меняем буферы
        ping = !ping;

        // Удваиваем размер блока
        run <<= 1;
    }

    // Остановка таймера
    CUDA_CHECK(cudaEventRecord(stop));
    CUDA_CHECK(cudaEventSynchronize(stop));

    // Время выполнения
    float ms = 0.0f;
    CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));

    // Определяем финальный буфер
    int* result = ping ? d0 : d1;

    // Копируем результат обратно на CPU
    CUDA_CHECK(cudaMemcpy(a.data(), result, bytes, cudaMemcpyDeviceToHost));

    // Освобождаем ресурсы
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d0);
    cudaFree(d1);

    // Возвращаем время GPU
    return ms;
}

// CPU-сортировка (для проверки)
double cpu_sort(vector<int>& a) {
    auto t1 = chrono::high_resolution_clock::now();
    stable_sort(a.begin(), a.end());
    auto t2 = chrono::high_resolution_clock::now();
    return chrono::duration<double, milli>(t2 - t1).count();
}

// Главная функция
int main() {

    // Размеры массивов по заданию
    vector<int> sizes = {10000, 100000};

    // Генератор случайных чисел
    mt19937 rng(42);
    uniform_int_distribution<int> dist(0, 1000000);

    // Размер чанка (1 блок = 1 подмассив)
    constexpr int BLOCK_SIZE = 1024;

    // Перебор размеров
    for (int n : sizes) {

        // Генерация исходного массива
        vector<int> base(n);
        for (int i = 0; i < n; i++)
            base[i] = dist(rng);

        // Копии для CPU и GPU
        vector<int> cpuArr = base;
        vector<int> gpuArr = base;

        // CPU сортировка
        double cpu_ms = cpu_sort(cpuArr);

        // GPU сортировка
        double gpu_ms = gpu_merge_sort<BLOCK_SIZE>(gpuArr);

        // Проверка корректности
        bool ok = (cpuArr == gpuArr);

        // Вывод результатов
        cout << "N = " << n << "\n";
        cout << "CPU stable_sort: " << cpu_ms << " ms\n";
        cout << "GPU merge sort:  " << gpu_ms << " ms\n";
        cout << "Correct: " << (ok ? "YES" : "NO") << "\n";
        if (gpu_ms > 0)
            cout << "Speedup CPU/GPU: " << cpu_ms / gpu_ms << "x\n";
    }

    return 0;
}


Writing 4main.cu


In [3]:
!nvcc -O2 -arch=sm_75 4main.cu -o 4main #компиляция CUDA-программы
!./4main #запуск программы

N = 10000
CPU stable_sort: 0.708077 ms
GPU merge sort:  0.534144 ms
Correct: YES
Speedup CPU/GPU: 1.32563x
N = 100000
CPU stable_sort: 9.58632 ms
GPU merge sort:  3.49811 ms
Correct: YES
Speedup CPU/GPU: 2.74043x
